**Phan Lê Anh Thư**

- Kiểm định tỉ lệ chuyển đổi của 2 mẫu landing page bằng phương pháp binomial test. (Notebook này)

In [1]:
import pandas as pd
import numpy as np
import datetime
from scipy.stats import chi2_contingency, beta
from IPython.display import Image

### 1. Mô tả thí nghiệm

Công ty phát triển một landing page mới và muốn test ảnh hưởng của 2 giao diện đến tỉ lệ chuyện đổi. Người dùng được chia làm 2 groups:

1. **Control**: Dùng bản cũ (old_page)
2. **Test/Treament**: Dùng bản mới (new_page)

**Tỉ lệ chuyển đổi (Conversion rate) = % (# Converted Users/# Exposed Users)**

Dự án chạy trong vòng 3 tuần, track liên tục data của người dùng về 2 hành động:
1. **Exposure**: Một người dùng truy cập trang. Người dùng được phân loại vào một trong 2 groups và ghi lại các trang liên quan trong lần đầu ghé page theo timestamp.
2. **Conversion**: Người dùng đã exposed trong vòng 7 ngày thực hiện giao dịch mua hàng.

### 2. Data Collection

A/B testing data: https://www.kaggle.com/saraabdelaal/abtestdata

In [2]:
df = pd.read_csv('ab_data.csv')

In [3]:
df.shape

(294478, 5)

In [4]:
df.sample(2)

,user_id,timestamp,group,landing_page,converted
191259,789201,2017-01-05 14:19:56.295213,control,old_page,0
198847,895168,2017-01-24 11:33:29.798136,control,old_page,0


Mỗi hàng dữ liệu chứ một lượt expose của user_id
- **timestamp**: thời gian
- **group**: nhóm
- **landing_page**: bản page mà họ truy cập (old_page hay new_page, tương ứng với group)
- **converted**: Mặc định là 0, chuyển thành 1 nếu họ convert trong 7 ngày kể từ lần expose đầu tiên.

In [5]:
start_time = datetime.datetime.strptime(df['timestamp'].min(), '%Y-%m-%d %H:%M:%S.%f')
end_time = datetime.datetime.strptime(df['timestamp'].max(), '%Y-%m-%d %H:%M:%S.%f')
data_duration = (end_time - start_time).days

print(f"Number of unique users in experiment: {df['user_id'].nunique()}")
print(f"Data collected for {data_duration} days")
print(f"Landing pages to compare: {df['landing_page'].unique().tolist()}")
print(f"Percentage of users in control: {round(df[df['group']=='control'].shape[0] * 100 / df.shape[0])}%")

Number of unique users in experiment: 290584
Data collected for 21 days
Landing pages to compare: ['old_page', 'new_page']
Percentage of users in control: 50%


Tổng quan về thử nghiệm:
- Có 290584 người sử dụng tham gia
- Diễn ra trong vòng 21 ngày
- Chạy trên 2 mẫu landing page (group): old_page(control group) và new_page(treatment group)
- Chia đều user vào 2 group (50/50 trong mỗi group)

### 3. Data Processing

**Xử lí các user_id lặp lại nhiều lần ở các group khác nhau**

In [6]:
sample = df[df['user_id'].isin([746755,722274])]
sample

,user_id,timestamp,group,landing_page,converted
29073,746755,2017-01-11 01:28:57.083669,control,new_page,1
105487,722274,2017-01-19 01:46:53.093257,control,old_page,0
262554,722274,2017-01-09 21:21:23.638444,control,new_page,0
286566,746755,2017-01-05 03:40:08.457451,control,old_page,0


Chú ý là user có id 722274 lặp lại 2 lần ở 2 group khác nhau, điều này không thỏa điều kiệu của testing. 
Với các user lặp lại này ta có thể: 
1. chỉ tính lần truy cập đầu tiên theo timestamp;
2. xem xét phương án lọc hẳn các user này khỏi dataset.

In [7]:
# 1. Get timestamp of first exposure
first_conversion = sample.groupby('user_id')['timestamp'].min().to_frame().reset_index()
sample = sample.merge(first_conversion, on=['user_id', 'timestamp'])
sample

,user_id,timestamp,group,landing_page,converted
0,722274,2017-01-09 21:21:23.638444,control,new_page,0
1,746755,2017-01-05 03:40:08.457451,control,old_page,0


In [8]:
counter = df['user_id'].value_counts()
(counter > 1).value_counts()

False    286690
True       3894
Name: user_id, dtype: int64

3894/290584 (tương ứng khoảng 1.34%) user_id tiếp cận cả 2 group. Vì tỉ lệ khá nhỏ, ta lọc tất cả các record của các user_id này khỏi dataset.

In [9]:
#2. Remove users with multiple buckets
valid_users = pd.DataFrame(counter[counter == 1].index, columns=['user_id'])
df = df.merge(valid_users, on=['user_id'])

### 4. Binomial test

In [10]:
# Calculate conversion rate

control = df[df['group']=='control']
treatment = df[df['group']=='treatment']

control_conversion_perc = round(control['converted'].sum() * 100/ control['converted'].count(), 3)
treatment_conversion_perc = round(treatment['converted'].sum() * 100/ treatment['converted'].count(), 3)
lift = round(treatment_conversion_perc - control_conversion_perc, 3)

print(f"Treatment Conversion Rate: {treatment_conversion_perc}%")
print(f"Control Conversion Rate: {control_conversion_perc}%")
print(f"Lift = {lift}%")

Treatment Conversion Rate: 11.873%
Control Conversion Rate: 12.017%
Lift = -0.144%


In [11]:
# Compare the conversion rate

n_sample_test = df['user_id'].nunique()
conversion_rate_test = treatment_conversion_perc/100 # Var_1
conversion_rate_control = control_conversion_perc/100 # Var_0

# What is the hypothesis?
# H0: r0 = r1
# Ha: r0 < r1

In [12]:
# Should we deploy the test design? -> Co su dung ket qua khong?

from scipy.stats import binomtest
k = int(conversion_rate_test * n_sample_test) # Conversion rate of test, used for binomtest
binomtest(k=k, n=n_sample_test, p=conversion_rate_control, alternative='greater')

# alternative: two-sided #, greator < , less >

BinomTestResult(k=34038, n=286690, alternative='greater', proportion_estimate=0.11872754543234854, pvalue=0.9913797085486543)

pvalue=0.9913797085486543 >> alpha = 0.05

Kết luận: không bác bỏ H0, không có bằng chứng thống kê cho thấy mẫu landing page mới tạo tỉ lệ chuyển đổi mua hàng lớn hơn mẫu cũ.

---
**Tham khảo: https://github.com/ajhalthor/bayesian-testing/**